In [1]:
from fpga_implementation import DESEncryptor

In [13]:
encryptor = DESEncryptor(max_parallel_send=2 ** 8)

# Example
message = "AAAAAAA" # 7 bytes, will be padded
key = "BBBBBBBB"     # 8 bytes
encrypted_result = encryptor.encrypt(message, key)
print(f"Encrypted result: {encrypted_result}")

message_long = "AAAAAAAAAAAAAAA" # 15 bytes
encrypted_result_long = encryptor.encrypt(message_long, key)
print(f"Encrypted result (long): {encrypted_result_long}")


Now sending part(0, 0): b'AAAAAAA\x00', 4141414141414100, with key: b'BBBBBBBB', 4242424242424242
Encrypted result: fd87106c0692ca94
Now sending part(0, 0): b'AAAAAAAA', 4141414141414141, with key: b'BBBBBBBB', 4242424242424242
Now sending part(1, 8): b'AAAAAAA\x00', 4141414141414100, with key: b'BBBBBBBB', 4242424242424242
Encrypted result (long): 57fa2ec1bb39ca18fd87106c0692ca94


In [14]:
from pydes_implementation import PyDES

In [15]:
# Test Cases
def test_compare_des_encrypt():
    """
    Test to compare the output of the PYNQ DES implementation with the pyDes library.
    """
    fpga_encryptor = DESEncryptor() 
    pyDes_encryptor = PyDES()

    test_vectors = [
        {"message": "ABCDEFGH", "key": "12345678"},
        {"message": "ABCDEFGHabcdefgh", "key": "12345678"},
        {"message": "ABCDEFG", "key": "12345678"},
        {"message": "", "key": "12345678"},
        {"message": "ABCDEFGHIJKLMNOPQRSTUVWXYZ123456", "key": "12345678"},
    ]

    for test_vector in test_vectors:
        message = test_vector["message"]
        key = test_vector["key"]
        print(f"Message: {message}, Key: {key}")

        # Get results from both implementations
        pynq_result_bytes = fpga_encryptor.encrypt(message, key)
        print(f"PYNQ Result (bytes): {pynq_result_bytes.hex()}")

        pyDes_result_bytes = pyDes_encryptor.encrypt(message, key)
        print(f"pyDes Result (bytes): {pyDes_result_bytes.hex()}")

        # Compare the results.  Note that the hex representation should be the same.
        assert pynq_result_bytes == pyDes_result_bytes, f"Results do not match for message: {message}, key: {key}"

test_compare_des_encrypt()


Using 16x type with input: 128 bits, and output: 64 bits
Message: ABCDEFGH, Key: 12345678
Now sending part(0, 0): b'ABCDEFGH', 4142434445464748, with key: b'12345678', 3132333435363738
PYNQ Result (bytes): 96de603eaed6256f
pyDes Result (bytes): 96de603eaed6256f
Message: ABCDEFGHabcdefgh, Key: 12345678
Now sending part(0, 0): b'ABCDEFGH', 4142434445464748, with key: b'12345678', 3132333435363738
Now sending part(1, 8): b'abcdefgh', 6162636465666768, with key: b'12345678', 3132333435363738
PYNQ Result (bytes): 96de603eaed6256f94d4436bc3b5b693
pyDes Result (bytes): 96de603eaed6256f94d4436bc3b5b693
Message: ABCDEFG, Key: 12345678
Now sending part(0, 0): b'ABCDEFG\x00', 4142434445464700, with key: b'12345678', 3132333435363738
PYNQ Result (bytes): 10565b3a6fd1aac7
pyDes Result (bytes): 10565b3a6fd1aac7
Message: , Key: 12345678
PYNQ Result (bytes): 
pyDes Result (bytes): 
Message: ABCDEFGHIJKLMNOPQRSTUVWXYZ123456, Key: 12345678
Now sending part(0, 0): b'ABCDEFGH', 4142434445464748, with key:

In [17]:
# Test Cases
def test_compare_des_decrypt():
    """
    Test to compare the output of the PYNQ DES implementation with the pyDes library.
    """
    fpga_encryptor = DESEncryptor() 
    pyDes_encryptor = PyDES()

    test_vectors = [
        {"message": "ABCDEFGH", "key": "12345678"},
        {"message": "ABCDEFGHabcdefgh", "key": "12345678"},
        {"message": "ABCDEFG", "key": "12345678"},
        {"message": "", "key": "12345678"},
        {"message": "ABCDEFGHIJKLMNOPQRSTUVWXYZ123456", "key": "12345678"},
    ]

    for test_vector in test_vectors:
        message = test_vector["message"]
        key = test_vector["key"]
        print(f"Message: {message}, Key: {key}")

        # Get results from both implementations
        pynq_result_bytes = fpga_encryptor.decrypt(message, key)
        print(f"PYNQ Result (bytes): {pynq_result_bytes.hex()}")

        pyDes_result_bytes = pyDes_encryptor.decrypt(message, key)
        print(f"pyDes Result (bytes): {pyDes_result_bytes.hex()}")

        # Compare the results.  Note that the hex representation should be the same.
        assert pynq_result_bytes == pyDes_result_bytes, f"Results do not match for message: {message}, key: {key}"
        

test_compare_des_decrypt()


Using 16x type with input: 128 bits, and output: 64 bits
Message: ABCDEFGH, Key: 12345678
Now sending part(0, 0): b'ABCDEFGH', 4142434445464748, with key: b'12345678', 3132333435363738
PYNQ Result (bytes): 05288ad59e6bc15f
pyDes Result (bytes): 05288ad59e6bc15f
Message: ABCDEFGHabcdefgh, Key: 12345678
Now sending part(0, 0): b'ABCDEFGH', 4142434445464748, with key: b'12345678', 3132333435363738
Now sending part(1, 8): b'abcdefgh', 6162636465666768, with key: b'12345678', 3132333435363738
PYNQ Result (bytes): 05288ad59e6bc15f22b7bf958db84680
pyDes Result (bytes): 05288ad59e6bc15f22b7bf958db84680
Message: ABCDEFG, Key: 12345678
Now sending part(0, 0): b'ABCDEFG\x00', 4142434445464700, with key: b'12345678', 3132333435363738
PYNQ Result (bytes): a6d49bebbeb2f6a5
pyDes Result (bytes): a6d49bebbeb2f6a5
Message: , Key: 12345678
PYNQ Result (bytes): 
pyDes Result (bytes): 
Message: ABCDEFGHIJKLMNOPQRSTUVWXYZ123456, Key: 12345678
Now sending part(0, 0): b'ABCDEFGH', 4142434445464748, with key:

In [8]:
# Performance

def compare_des_timing():
    """
    Compares the execution time of the PYNQ DES implementation with the pyDes library.
    """

    test_vectors = [
        {"message": "ABCDEFGH", "key": "12345678"},
        {"message": "ABCDEFGHabcdefgh", "key": "12345678"},
        {"message": "ABCDEFG", "key": "12345678"},
        {"message": "", "key": "12345678"},
        {"message": "ABCDEFGHIJKLMNOPQRSTUVWXYZ123456", "key": "12345678"},
        {"message": "ABCDEFGHIJKLMNOPQRSTUVWXYZ123456ABCDEFGHIJKLMNOPQRSTUVWXYZ123456", "key": "12345678"},
        {"message": "A" * 128, "key": "12345678"},
        {"message": "A" * 128 * 4, "key": "12345678"},
#         {"message": "A" * 2 ** 20 * 8, "key": "12345678"},
    ]
    
    
    largest_size = max(map(lambda x: len(x["message"]), test_vectors)) // 8
    print("Instanting DES encryptor with size: ", largest_size)
    
    encryptor = DESEncryptor(logging=False, max_parallel_send=largest_size)  # Create an instance of DESEncryptor
    

    num_iterations = 10  # Number of times to repeat the encryption for timing

    print("Timing Comparison:")
    print(f"Number of iterations: {num_iterations}")

    for test_vector in test_vectors:
        message = test_vector["message"]
        key = test_vector["key"]

        # Time PYNQ encryption
        start_time_pynq = time.time()
        for _ in range(num_iterations):
            pynq_result_hex = encryptor.encrypt(message, key)
        end_time_pynq = time.time()
        pynq_time = end_time_pynq - start_time_pynq

        # Time pyDes encryption
        start_time_pydes = time.time()
        for _ in range(num_iterations):
            pyDes_result_bytes = encrypt_with_pyDes(message, key)
        end_time_pydes = time.time()
        pydes_time = end_time_pydes - start_time_pydes

        print(f"\nMessage: {message}, Key: {key}")
        print(f"PYNQ Time: {pynq_time:.6f} seconds")
        print(f"pyDes Time: {pydes_time:.6f} seconds")
        print(f"Speed up of: {pydes_time / pynq_time:.6f}")

        # Compare the results (optional, for functional verification)
        pynq_result_bytes = bytes.fromhex(pynq_result_hex)
        pyDes_result_bytes = encrypt_with_pyDes(message, key)
        assert pynq_result_bytes == pyDes_result_bytes, f"Results do not match for message: {message}, key: {key}"

compare_des_timing()
print("Done")

Instanting DES encryptor with size:  64
Using 64x type with input: 128 bits, and output: 64 bits
Timing Comparison:
Number of iterations: 10

Message: ABCDEFGH, Key: 12345678
PYNQ Time: 0.012247 seconds
pyDes Time: 0.087132 seconds
Speed up of: 7.114765

Message: ABCDEFGHabcdefgh, Key: 12345678
PYNQ Time: 0.011653 seconds
pyDes Time: 0.151223 seconds
Speed up of: 12.977003

Message: ABCDEFG, Key: 12345678
PYNQ Time: 0.011963 seconds
pyDes Time: 0.091754 seconds
Speed up of: 7.669576

Message: , Key: 12345678
PYNQ Time: 0.010803 seconds
pyDes Time: 0.030572 seconds
Speed up of: 2.829850

Message: ABCDEFGHIJKLMNOPQRSTUVWXYZ123456, Key: 12345678
PYNQ Time: 0.012359 seconds
pyDes Time: 0.265650 seconds
Speed up of: 21.494213

Message: ABCDEFGHIJKLMNOPQRSTUVWXYZ123456ABCDEFGHIJKLMNOPQRSTUVWXYZ123456, Key: 12345678
PYNQ Time: 0.013435 seconds
pyDes Time: 0.508620 seconds
Speed up of: 37.858119

Message: AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA